In [1]:
#!pip install HASS-data-detective # Install detective

In [2]:
!pip show HASS-data-detective

Name: HASS-data-detective
Version: 3.0
Summary: Tools for studying Home Assistant data.
Home-page: https://github.com/robmarkcole/HASS-data-detective
Author: Robin Cole
Author-email: robmarkcole@gmail.com
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: pandas, pytz, ruamel.yaml, SQLAlchemy
Required-by: 


In [3]:
import detective.core as detective
import detective.functions as functions
import pandas as pd

db = detective.db_from_hass_config()

YAML tag !include_dir_merge_named is not supported
Successfully connected to database sqlite:////config/home-assistant_v2.db
There are 464 entities with data


In [4]:
dir(db)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'con',
 'db_type',
 'engine',
 'entities',
 'fetch_all_data_of',
 'fetch_all_sensor_data',
 'fetch_entities',
 'perform_query',
 'url']

In [5]:
df = db.fetch_all_sensor_data()


            SELECT states.state, states.last_updated_ts, states_meta.entity_id
            FROM states
            JOIN states_meta
            ON states.metadata_id = states_meta.metadata_id
            WHERE
                states_meta.entity_id  LIKE '%sensor%'
            AND
                states.state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated_ts DESC
        LIMIT 50000
The returned Pandas dataframe has 50000 rows of data.


In [6]:
db.entities

['automation.cerpadlo_vratky_tuv_aktivovat_pro_dnesek',
 'automation.cerpadlo_vratky_tuv_deaktivovat_pres_noc',
 'automation.chlazeni_stridac',
 'automation.energy_to_tuv',
 'automation.energy_to_tuv_stop',
 'automation.force_heating_on_cheap_spot',
 'automation.force_tuv_heating_on_low_spot',
 'automation.klimatizace_stridac_stop',
 'automation.odvlhcovac_test',
 'automation.on_cheap_spot_force_tuv_heating',
 'automation.on_cheap_spot_leaving_set_back_tuv_heating',
 'automation.on_cheap_spot_leaving_set_heating_back',
 'automation.on_expensive_spot_turn_the_tuv_off',
 'automation.pravidelne_spousteni_cerpadla_vratky_tuv',
 'automation.spot_limit_negative_price',
 'automation.spustit_prodej_elektriny',
 'automation.turn_off_heating_and_tuv_for_most_expensive_spot',
 'automation.turn_on_heating_and_tuv_when_spot_price_is_not_so_high',
 'automation.turn_on_the_heating_on_low_spot_price',
 'automation.vypnuti_cerpadla_vratky_po_par_minutach',
 'binary_sensor.dvere_spojovaci_cho_battery_lo

In [7]:

df = db.fetch_all_data_of([
    # 'sensor.total_energy_export',
    'weather.forecast_home'
], 
                         # get_attributes=True
                         )
df

TypeError: HassDatabase.fetch_all_data_of() got an unexpected keyword argument 'get_attributes'

In [ ]:
help(db.fetch_all_data_of)

In [14]:
from sqlalchemy import create_engine, text
from typing import Tuple
import json

def fetch_all_data_of(self, sensors: Tuple[str], limit=50000, get_attributes=True, parse_attributes=True, print_query=False) -> pd.DataFrame:
    sensors_str = str(tuple(sensors))
    if len(sensors) == 1:
        sensors_str = sensors_str.replace(",", "")

    query_no_attr = f"""
        WITH combined_states AS (
            SELECT states.state, states.last_updated_ts, states_meta.entity_id
            FROM states
            JOIN states_meta
            ON states.metadata_id = states_meta.metadata_id
        )
        SELECT *
        FROM combined_states
        WHERE 
            entity_id IN {sensors_str}
        AND
            state NOT IN ('unknown', 'unavailable')
        ORDER BY last_updated_ts DESC
    """

    query_attr = f"""
        WITH combined_states AS (
            SELECT states.state, states.last_updated_ts, states_meta.entity_id, state_attributes.shared_attrs AS attributes
            FROM states
            JOIN states_meta
            ON states.metadata_id = states_meta.metadata_id
            LEFT JOIN state_attributes
            ON states.attributes_id = state_attributes.attributes_id
        )
        SELECT *
        FROM combined_states
        WHERE 
            entity_id IN {sensors_str}
        AND
            state NOT IN ('unknown', 'unavailable')
        ORDER BY last_updated_ts DESC
    """

    query = query_attr if get_attributes else query_no_attr

    if limit is not None:
        query += f"LIMIT {limit}"
    if print_query:
        print(query)
    query = text(query)
    df = pd.read_sql_query(query, con=self.con)

       # Parse attributes if they exist
    if parse_attributes:
        if 'attributes' in df.columns:
            df['parsed_attributes'] = df['attributes'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})

    return df


In [15]:

df = fetch_all_data_of(db, [
    # 'sensor.total_energy_export',
    'weather.forecast_home'
])
df

,state,last_updated_ts,entity_id,attributes,parsed_attributes
0,partlycloudy,1.731184e+09,weather.forecast_home,"{""temperature"":3.1,""dew_point"":1.7,""temperatur...","{'temperature': 3.1, 'dew_point': 1.7, 'temper..."
1,partlycloudy,1.731183e+09,weather.forecast_home,"{""temperature"":3.1,""dew_point"":1.7,""temperatur...","{'temperature': 3.1, 'dew_point': 1.7, 'temper..."
2,partlycloudy,1.731180e+09,weather.forecast_home,"{""temperature"":2.3,""dew_point"":1.0,""temperatur...","{'temperature': 2.3, 'dew_point': 1.0, 'temper..."
3,cloudy,1.731177e+09,weather.forecast_home,"{""temperature"":3.4,""dew_point"":2.0,""temperatur...","{'temperature': 3.4, 'dew_point': 2.0, 'temper..."
4,cloudy,1.731173e+09,weather.forecast_home,"{""temperature"":3.7,""dew_point"":2.4,""temperatur...","{'temperature': 3.7, 'dew_point': 2.4, 'temper..."
...,...,...,...,...,...
243,fog,1.730275e+09,weather.forecast_home,"{""temperature"":10.6,""dew_point"":10.0,""temperat...","{'temperature': 10.6, 'dew_point': 10.0, 'temp..."
244,fog,1.730271e+09,weather.forecast_home,"{""temperature"":10.7,""dew_point"":10.3,""temperat...","{'temperature': 10.7, 'dew_point': 10.3, 'temp..."
245,fog,1.730267e+09,weather.forecast_home,"{""temperature"":11.2,""dew_point"":11.0,""temperat...","{'temperature': 11.2, 'dew_point': 11.0, 'temp..."
246,cloudy,1.730264e+09,weather.forecast_home,"{""temperature"":11.5,""dew_point"":11.1,""temperat...","{'temperature': 11.5, 'dew_point': 11.1, 'temp..."


In [18]:
df.parsed_attributes[0]["temperature"]

3.1

In [19]:
df['temperature'] = df['parsed_attributes'].apply(lambda x: x.get('temperature') if isinstance(x, dict) else None)

In [20]:
df

,state,last_updated_ts,entity_id,attributes,parsed_attributes,temperature
0,partlycloudy,1.731184e+09,weather.forecast_home,"{""temperature"":3.1,""dew_point"":1.7,""temperatur...","{'temperature': 3.1, 'dew_point': 1.7, 'temper...",3.1
1,partlycloudy,1.731183e+09,weather.forecast_home,"{""temperature"":3.1,""dew_point"":1.7,""temperatur...","{'temperature': 3.1, 'dew_point': 1.7, 'temper...",3.1
2,partlycloudy,1.731180e+09,weather.forecast_home,"{""temperature"":2.3,""dew_point"":1.0,""temperatur...","{'temperature': 2.3, 'dew_point': 1.0, 'temper...",2.3
3,cloudy,1.731177e+09,weather.forecast_home,"{""temperature"":3.4,""dew_point"":2.0,""temperatur...","{'temperature': 3.4, 'dew_point': 2.0, 'temper...",3.4
4,cloudy,1.731173e+09,weather.forecast_home,"{""temperature"":3.7,""dew_point"":2.4,""temperatur...","{'temperature': 3.7, 'dew_point': 2.4, 'temper...",3.7
...,...,...,...,...,...,...
243,fog,1.730275e+09,weather.forecast_home,"{""temperature"":10.6,""dew_point"":10.0,""temperat...","{'temperature': 10.6, 'dew_point': 10.0, 'temp...",10.6
244,fog,1.730271e+09,weather.forecast_home,"{""temperature"":10.7,""dew_point"":10.3,""temperat...","{'temperature': 10.7, 'dew_point': 10.3, 'temp...",10.7
245,fog,1.730267e+09,weather.forecast_home,"{""temperature"":11.2,""dew_point"":11.0,""temperat...","{'temperature': 11.2, 'dew_point': 11.0, 'temp...",11.2
246,cloudy,1.730264e+09,weather.forecast_home,"{""temperature"":11.5,""dew_point"":11.1,""temperat...","{'temperature': 11.5, 'dew_point': 11.1, 'temp...",11.5


In [ ]:
# convert last_updated_ts to datetime
df['last_updated_datetime'] = pd.to_datetime(df['last_updated_ts'], unit='s')


In [1]:
#round all to hours and get the mean of temperature
df['last_updated_datetime'] = df['last_updated_datetime'].dt.floor('H')

NameError: name 'df' is not defined

In [ ]:
# make mean of temperature with the same last_updated_datetime
df = df.groupby('last_updated_datetime').agg({'temperature':'mean'}).reset_index()
# order by last_updated_datetime
df = df.sort_values('last_updated_datetime')

In [ ]:
import seaborn as sns

sns.lineplot(data=df, x='last_updated_datetime', y='temperature')

In [ ]:
df = fetch_all_data_of(db, [
    # 'sensor.total_energy_export',
    'sensor.house_consumption'
])
df['last_updated_ts'] = pd.to_datetime(df['last_updated_ts'], unit='s')
df
df.state = df.state.astype(int)

In [ ]:
# house_consumption is in Watts. I want to integrate it to get the energy consumption in kWh
import numpy as np
df['energy'] = np.trapz(y=df['state'], x=df['last_updated_ts'].astype(int)/1e9/3600)

